In [198]:
import pandas as pd
import numpy as np

# Reading data from csv

In [380]:
data_Singapore = pd.read_csv("data/SingaporeTrain.csv")
data_NY = pd.read_csv("data/NYTrain.csv")
data_London = pd.read_csv("data/LondonTrain.csv")
data_Singapore.head()
# data_NY.head()
# data_London.head()

,row ID,educationInfoForAgeGroupEstimation,workInfoForAgeGroupEstimation,gender,realAge,ageGroup,relationship,educationLevel,occupation,income
0,d0cc2d6caf6eb55ccf606770f24df2b5,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN
1,eb4e2aa0bf687227821b17cfcd5365e3,Bukit Batok Secondary School ITE College West ...,NaN,female,NaN,NaN,NaN,college,NaN,NaN
2,acfb8e93139fe820996b27ee3c6bc9db,"St Patrick&#039;s School, Singapore Class of 2...",NaN,male,NaN,AGE10_20,single,college,NaN,NaN
3,ea251951bd3c7667daf214e8efa92f4e,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,$$
4,f8f7b497ad8167535345bfc3d59d6aaa,NaN,PT. Duta Marga Lestarindo September 2013 to pr...,female,NaN,NaN,NaN,NaN,archetecture and engineering,NaN


# Combining all the data to a single dataframe

In [414]:
data_Singapore['source'] = 'Singapore'
data_NY['source'] = 'NY'
data_London['source'] = 'London'
data = pd.concat([data_Singapore, data_NY, data_London],ignore_index=True)
data.shape

(10241, 11)

In [604]:
data['source'].unique()

array(['Singapore', 'NY', 'London'], dtype=object)

In [383]:
data['gender'].unique()

array(['female', 'male', nan], dtype=object)

In [384]:
data['realAge'].unique()

array([nan, 27., 19., 20., 22., 21., 34., 31., 23., 18., 36., 24., 26.,
       29., 30., 25., 56., 46., 33., 28., 82., 35., 37., 39.])

In [385]:
data['ageGroup'].unique()

array([nan, 'AGE10_20', 'AGE20_30', 'AGE30_40', 'AGE40_50', 'AGE50_INF'],
      dtype=object)

In [386]:
data['relationship'].unique()

array([nan, 'single', 'in a relationship', 'married'], dtype=object)

In [387]:
data['educationLevel'].unique()

array([nan, 'college', 'school', 'undergraduate', 'graduate'],
      dtype=object)

In [388]:
data['occupation'].unique()

array([nan, 'archetecture and engineering',
       'food preparation and service related',
       'arts, design, entertainment, sports, and media', 'management',
       'office and administrative support', 'personal care and service',
       'protective service', 'healthcare support', 'sales and related',
       'legal', 'transportation and material moving',
       'computer and mathematical', 'production',
       'life, physical, and social science',
       'education, training, and library',
       'healthcare practitioners and technical',
       'building and grounds cleaning and maintenance',
       'farming, fishing and forestry', 'construction and extraction',
       'business and financial operations',
       'community and social service'], dtype=object)

In [389]:
data['income'].unique()

array([nan, '$$', '$', '$$$', '$$$$'], dtype=object)

# Exctracting features from "workInfoForAgeGroupEstimation"

In [738]:
row_id = 4
row = data['workInfoForAgeGroupEstimation'][row_id]
# row = 'PT. Duta Marga Lestarindo September 19 present Land Transport Authority Project Engineer � August 2011 to July 2013 T.Y. Lin International Civil Engineer � January 2010 to June 2010'
row

'PT. Duta Marga Lestarindo September 2013 to present Land Transport Authority Project Engineer � August 2011 to July 2013 T.Y. Lin International Civil Engineer � January 2010 to June 2010'

In [739]:
def process_work_info(row):
    years_raw = re.compile(r'\b(?:19|20)\d{2}|\bpresent').findall(row)
    years =  [int(year) if year != 'present' else 2018 for year in years_raw]
    if len(years)<2:
        return None
    else:
#         print("Extracted years:",years)
        pairs = [years[i:i+2] for i in range(0, len(years)-1, 2)]
        lengths = [pair[1]-pair[0] for pair in pairs]
        number_of_places = len(pairs)
        max_length = max(lengths)
        min_length = min(lengths)
        total_length = sum(lengths)
        mean_length = total_length / float(len(lengths))
        start_year = min(years)
        working_now = None
        if len(years_raw)>0:
            working_now = 'present' in years_raw 
    #     print(start_year)
    #     print(number_of_places)
    #     print(total_length)
    #     print(mean_length)
    #     print(min_length)
    #     print(max_length)
    #     print(working_now)
    return start_year, number_of_places, total_length, mean_length, min_length, max_length, working_now

In [740]:
process_work_info(row)

(2010, 3, 7, 2.3333333333333335, 0, 5, True)

# Exctracting features from "educationInfoForAgeGroupEstimation"

In [ ]:
# possible_levels = ['college', 'school', 'undergraduate', 'graduate']

# possible_values_for_levels = {}
# "Secondary School", "Ngee", "Institute", "University", "High School"
# # CHIJ Secondary (Toa Payoh) - автономная католическая школа для девочек в Сингапуре. 
# Class of 2010

In [789]:
row_id = 45
row = data['educationInfoForAgeGroupEstimation'][row_id]
# row = "Republic Polytechnic 2012 to 2015 � Singapore CHIJ Kellock Singapore CHIJ St. Theresa&#039;s Convent Class of 2011 � Singapore Republic Polytechnic Integrated Event Management � Singapore"
row

'SMK Seri Tanjong Class of 2009 � Melaka sek. men. seri tanjung,melaka (PMR) Class of 2005 � Malacca City, Malaysia sek. men. seri tanjung,melaka (SPM) Class of 2007 Maktab Koperasi Malaysia Class of 2012 � Kuching, Malaysia Uni of Oxford Oxford, Oxfordshire skm seri tanjung Class of 2009 � Malacca City, Malaysia'

In [785]:
def process_education_info(row):
    class_of_years = [int(i) for i in re.compile(r'(?<=of )\d{4}').findall(row)]
    range_years_starts = [int(i) for i in re.compile(r'\b(?:19|20)\d{2}(?= to )').findall(row)]
    range_years_ends = [int(i) if i != 'present' else 2018 for i in re.compile(r'(?<= to )(?:19|20)\d{2}|\bpresent').findall(row)]

    first_start_year = None
    last_start_year = None
    if len(class_of_years)>0 or len(range_years_starts)>0:
        first_start_year = min(class_of_years+range_years_starts)
        last_start_year = max(class_of_years+range_years_starts) 

    
    finish_year = None
    if len(range_years_ends)>0:
        finish_year = max(range_years_ends)
    # else
    #     if len(class_of_years)>0 | len(range_years_starts)>0:
    #     finish_year = max(class_of_years) + 4 #usually one program = 4year

    # study_now = None

    number_of_programs = None
    num = len(class_of_years)+len(range_years_starts)
    if num<1 and len(row)>0:
        number_of_programs = 1
    else:
        number_of_programs = num

    return first_start_year, last_start_year, finish_year, number_of_programs

In [786]:
process_education_info(row)

(2005, 2012, None, 5)

# I think we should build classifier on 
//all text from education, work and occupation for .. prediction..

In [ ]:
# data.shape
# (10241, 11)
# data[data.realAge.isnull()].shape 
# (10039, 11)
# data[data.ageGroup.isnull()].shape 
# (9179, 11)

In [455]:
data[data.ageGroup.isnull()].shape 
# (9179, 11)
# [['realAge', 'ageGroup']][20:]

(9179, 11)